In [ ]:
# Run this once if the packages are not installed yet
# !pip install pillow umap-learn scikit-learn pandas matplotlib


In [1]:
import os
import re
import glob
import tarfile
import csv

from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap
from sklearn.manifold import trustworthiness
from sklearn.neighbors import NearestNeighbors

# Matches grayscale  "1x1", "32x18"
# and color pixels   "1x1_r", "1x1_g", "1x1_b"
PIXEL_RE = re.compile(r"^\d+x\d+(?:_[rgb])?$")

# Filename pattern for Edinburgh frames
FILENAME_RE = re.compile(
    r"inspacecam163_(\d{4})_(\d{2})_(\d{2})_(\d{2})_(\d{2})_(\d{2})\.jpg$"
)


In [2]:
def parse_time_from_filename(name):
    """Returns (hour, minute, second) or None if the filename does not match."""
    base = os.path.basename(name)
    m = FILENAME_RE.match(base)
    if not m:
        return None
    h = int(m.group(4))
    m_ = int(m.group(5))
    s = int(m.group(6))
    return h, m_, s


def seconds_since_midnight(h, m, s):
    return h * 3600 + m * 60 + s


def parse_time_string(s):
    """
    Parse HH:MM or HH:MM:SS -> seconds since midnight.
    """
    parts = s.split(":")
    if len(parts) == 2:
        h, m = parts
        s_val = 0
    elif len(parts) == 3:
        h, m, s_val = parts
    else:
        raise ValueError(f"Invalid time format: {s!r}. Expected HH:MM or HH:MM:SS")
    h_i = int(h)
    m_i = int(m)
    s_i = int(s_val)
    if not (0 <= h_i < 24 and 0 <= m_i < 60 and 0 <= s_i < 60):
        raise ValueError(f"Time out of range: {s!r}")
    return seconds_since_midnight(h_i, m_i, s_i)


def time_filter(day_sec, start_sec, end_sec, use_all_times):
    if use_all_times:
        return True
    return start_sec <= day_sec <= end_sec


In [3]:
def list_tar_files(folder):
    pattern = os.path.join(folder, "day_*.tar")
    tar_paths = sorted(glob.glob(pattern))
    if not tar_paths:
        raise SystemExit(f"No tar files matching 'day_*.tar' found in {folder}")
    return tar_paths


def first_pass_lengths(tar_paths, start_sec, end_sec, use_all_times, frame_step):
    """
    First pass: determine, for each tar, how many kept frames (after subsampling)
    fall into the specified time window (or all times if use_all_times is True).

    Returns:
      - line_index_by_path: {tar_path -> line_id}
      - line_length_by_path: {tar_path -> number_of_kept_frames}
      - global_max_step: maximum step index across all lines
    """
    line_index_by_path = {}
    line_length_by_path = {}
    line_lengths = []

    line_counter = 0

    for path in tar_paths:
        with tarfile.open(path, "r") as tar:
            times = []
            for member in tar.getmembers():
                if not member.isfile():
                    continue
                t = parse_time_from_filename(member.name)
                if t is None:
                    continue
                h, m, s = t
                day_sec = seconds_since_midnight(h, m, s)
                if time_filter(day_sec, start_sec, end_sec, use_all_times):
                    times.append(day_sec)

        times.sort()
        if times:
            kept_len = (len(times) + frame_step - 1) // frame_step
            line_index_by_path[path] = line_counter
            line_length_by_path[path] = kept_len
            line_lengths.append(kept_len)
            line_counter += 1

    if not line_lengths:
        raise SystemExit("No frames found in the specified time window.")

    max_len = max(line_lengths)
    global_max_step = max_len - 1 if max_len > 1 else 0

    return line_index_by_path, line_length_by_path, global_max_step


In [4]:
def build_header(width, height, grayscale):
    header = [
        "id",
        "line",
        "label",
        "step",
        "daytimestamp",
        "daytime",
        "action",
        "age",
        "age_global",
    ]
    # Pixel columns; grayscale: <x>x<y>
    # Color: <x>x<y>_r, <x>x<y>_g, <x>x<y>_b
    for x in range(1, width + 1):
        for y in range(1, height + 1):
            if grayscale:
                header.append(f"{x}x{y}")
            else:
                header.append(f"{x}x{y}_r")
                header.append(f"{x}x{y}_g")
                header.append(f"{x}x{y}_b")
    return header


def extract_pixels(img_fileobj, width, height, grayscale):
    """
    Open image, convert to target mode and size, and return a flat list of values.
    Grayscale: one value per pixel.
    Color: three values (R, G, B) per pixel.
    """
    img = Image.open(img_fileobj)
    if grayscale:
        img = img.convert("L")
    else:
        img = img.convert("RGB")
    img = img.resize((width, height), resample=Image.BILINEAR)

    pixels = img.load()
    values = []
    for x in range(width):         # x = 0..width-1
        for y in range(height):    # y = 0..height-1
            if grayscale:
                values.append(pixels[x, y])
            else:
                r, g, b = pixels[x, y]
                values.extend([r, g, b])
    return values


In [5]:
def second_pass_write_csv(
    tar_paths,
    line_index_by_path,
    line_length_by_path,
    global_max_step,
    start_sec,
    end_sec,
    use_all_times,
    frame_step,
    width,
    height,
    grayscale,
    output_path,
):
    header = build_header(width, height, grayscale)
    global_id = 0
    total_rows = 0

    with open(output_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)

        for path in tar_paths:
            if path not in line_index_by_path:
                continue  # no frames in window

            line_id = line_index_by_path[path]
            line_len = line_length_by_path[path]

            frames = []
            with tarfile.open(path, "r") as tar:
                for member in tar.getmembers():
                    if not member.isfile():
                        continue
                    t = parse_time_from_filename(member.name)
                    if t is None:
                        continue
                    h, m, s = t
                    day_sec = seconds_since_midnight(h, m, s)
                    if not time_filter(day_sec, start_sec, end_sec, use_all_times):
                        continue
                    time_str = f"{h:02d}:{m:02d}:{s:02d}"
                    frames.append((day_sec, time_str, member))

                frames.sort(key=lambda x: x[0])
                frames = frames[::frame_step]

                for step, (day_sec, time_str, member) in enumerate(frames):
                    if line_len > 1:
                        age = step / (line_len - 1)
                    else:
                        age = 0.0

                    if global_max_step > 0:
                        age_global = step / global_max_step
                    else:
                        age_global = 0.0

                    img_fileobj = tar.extractfile(member)
                    if img_fileobj is None:
                        continue

                    pixel_values = extract_pixels(
                        img_fileobj,
                        width=width,
                        height=height,
                        grayscale=grayscale,
                    )

                    row = [
                        global_id,          # id
                        line_id,            # line (trajectory/day id)
                        "",                 # label
                        step,               # step
                        day_sec,            # daytimestamp
                        time_str,           # daytime
                        "",                 # action
                        f"{age:.6f}",       # age
                        f"{age_global:.6f}" # age_global
                    ] + pixel_values

                    writer.writerow(row)
                    global_id += 1
                    total_rows += 1

    print(f"Wrote {total_rows} rows to {output_path}")


In [6]:
def make_trajectory_filename(
    output_dir,
    width,
    height,
    grayscale,
    use_all_times,
    start_time,
    end_time,
    frame_step,
):
    """
    Create a unique filename for trajectories based on key parameters.
    """
    mode = "gray" if grayscale else "rgb"
    if use_all_times:
        tw = "all"
    else:
        tw = f"{start_time.replace(':','')}-{end_time.replace(':','')}"
    name = f"trajectories_w{width}h{height}_{mode}_tw{tw}_step{frame_step}.csv"
    return os.path.join(output_dir, name)


def build_trajectories_csv(
    folder,
    output_dir=".",
    output_path=None,
    width=128,
    height=72,
    grayscale=True,
    start_time="15:00:00",
    end_time="15:06:59",
    use_all_times=False,
    frame_step=1,
):
    """
    Build trajectory CSV from Edinburgh office monitoring tar files.

    Each row = one frame (downsampled image).
    'line' encodes the day/trajectory id, 'step' encodes the time order within line.

    If output_path is None, a unique filename is generated from parameters.
    """
    if frame_step <= 0:
        raise ValueError("frame_step must be >= 1")

    os.makedirs(output_dir, exist_ok=True)

    if output_path is None:
        output_path = make_trajectory_filename(
            output_dir=output_dir,
            width=width,
            height=height,
            grayscale=grayscale,
            use_all_times=use_all_times,
            start_time=start_time,
            end_time=end_time,
            frame_step=frame_step,
        )
    else:
        out_dir = os.path.dirname(output_path)
        if out_dir:
            os.makedirs(out_dir, exist_ok=True)

    tar_paths = list_tar_files(folder)

    if use_all_times:
        start_sec = 0
        end_sec = 24 * 3600 - 1
    else:
        start_sec = parse_time_string(start_time)
        end_sec = parse_time_string(end_time)
        if end_sec < start_sec:
            raise ValueError("end_time must be >= start_time")

    (
        line_index_by_path,
        line_length_by_path,
        global_max_step,
    ) = first_pass_lengths(
        tar_paths,
        start_sec,
        end_sec,
        use_all_times,
        frame_step,
    )

    print(f"Found {len(line_index_by_path)} trajectories (days)")

    second_pass_write_csv(
        tar_paths,
        line_index_by_path,
        line_length_by_path,
        global_max_step,
        start_sec,
        end_sec,
        use_all_times,
        frame_step,
        width,
        height,
        grayscale,
        output_path,
    )

    return output_path


In [ ]:
# === GLOBAL CONFIG ===
SEED = 42  # used for sampling + UMAP

edinburgh_folder = "."   # <- adjust this path
trajectories_output_dir = "trajectories"      # where trajectory CSVs go
embeddings_output_dir   = "embeddings"        # where .npy embeddings and plots go

# Time window / resolution settings
width = 128
height = 72
grayscale = True       # False = keep RGB
use_all_times = False  # True = ignore time window
start_time = "15:00:00"
end_time   = "15:06:59"
frame_step = 1         # 10 = one frame every 10 frames, etc.

os.makedirs(trajectories_output_dir, exist_ok=True)
os.makedirs(embeddings_output_dir, exist_ok=True)

# Build trajectories CSV (auto-named based on parameters)
traj_csv_path = build_trajectories_csv(
    folder=edinburgh_folder,
    output_dir=trajectories_output_dir,
    width=width,
    height=height,
    grayscale=grayscale,
    start_time=start_time,
    end_time=end_time,
    use_all_times=use_all_times,
    frame_step=frame_step,
)

print("Trajectory CSV ready:", traj_csv_path)


SystemExit: No tar files matching 'day_*.tar' found in /path/to/edinburgh_tars

c:\Users\Christian\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
df = pd.read_csv(traj_csv_path)
print("Full dataframe shape:", df.shape)
df.head()


In [ ]:
def find_pixel_columns(df: pd.DataFrame):
    pixel_cols = [c for c in df.columns if PIXEL_RE.match(c)]
    if not pixel_cols:
        raise SystemExit(
            "No pixel columns found. Expected columns like '1x1', '1x2', "
            "'1x1_r', etc."
        )
    return pixel_cols


def sample_frames(df: pd.DataFrame, max_samples=None, random_state=0):
    """
    Optional subsampling for UMAP experiments, to keep runs interactive.
    """
    if max_samples is not None and len(df) > max_samples:
        return (
            df.sample(n=max_samples, random_state=random_state)
            .sort_values(["line", "step"])
            .reset_index(drop=True)
        )
    return df.reset_index(drop=True)


# Example: work on a subset for quick experimentation
max_samples = 8000   # or None for all
df_sub = sample_frames(df, max_samples=max_samples, random_state=SEED)

pixel_cols = find_pixel_columns(df_sub)
X = df_sub[pixel_cols].to_numpy(dtype=np.float32)

print("Subsampled frames:", len(df_sub))
print("Pixel dim:", X.shape[1])


In [ ]:
def compute_quality_metrics(X_high, X_low, n_neighbors=15):
    """
    Compute simple projection quality metrics:
      - trustworthiness
      - kNN overlap (Jaccard over neighbor sets)
    """
    n_samples = X_high.shape[0]
    if n_samples <= 1:
        return {"trustworthiness": np.nan, "knn_overlap": np.nan}

    k = min(n_neighbors, n_samples - 1)

    # Trustworthiness (sklearn implementation)
    tw = trustworthiness(X_high, X_low, n_neighbors=k)

    # kNN overlap
    nn_high = NearestNeighbors(n_neighbors=k + 1, metric="euclidean").fit(X_high)
    nn_low = NearestNeighbors(n_neighbors=k + 1, metric="euclidean").fit(X_low)

    _, idx_high = nn_high.kneighbors(X_high)
    _, idx_low = nn_low.kneighbors(X_low)

    overlaps = []
    for i in range(n_samples):
        hi = set(idx_high[i, 1:])  # drop self
        lo = set(idx_low[i, 1:])
        inter = len(hi & lo)
        union = len(hi | lo)
        if union > 0:
            overlaps.append(inter / union)

    knn_overlap = float(np.mean(overlaps)) if overlaps else np.nan

    return {
        "trustworthiness": float(tw),
        "knn_overlap": knn_overlap,
    }


In [ ]:
def make_embedding_filename(
    output_dir,
    traj_csv_path,
    seed,
    n_neighbors,
    min_dist,
    metric,
    n_samples,
):
    """
    Create a unique filename for an embedding based on:
    - base trajectory CSV
    - sample count
    - UMAP params
    - seed
    """
    os.makedirs(output_dir, exist_ok=True)
    base_traj = os.path.splitext(os.path.basename(traj_csv_path))[0]
    md_str = str(min_dist).replace(".", "p")
    metric_str = metric.replace(" ", "_")
    name = (
        f"{base_traj}_samples{n_samples}_metric{metric_str}"
        f"_nn{n_neighbors}_md{md_str}_seed{seed}.npy"
    )
    return os.path.join(output_dir, name)


def run_umap_grid(
    X,
    n_neighbors_list,
    min_dist_list,
    metric,
    random_state,
    quality_k,
    traj_csv_path,
    embeddings_dir,
):
    """
    Run UMAP for all (n_neighbors, min_dist) combinations.

    For each combo:
      - tries to load an existing embedding from disk (based on params + seed)
      - otherwise runs UMAP, saves the embedding to .npy, and records metrics.

    Returns:
      - results_df: one row per combo with metrics
      - embeddings: {(n_neighbors, min_dist) -> (N, 2) array}
    """
    os.makedirs(embeddings_dir, exist_ok=True)

    results = []
    embeddings = {}
    n_samples = X.shape[0]

    for nn in n_neighbors_list:
        for md in min_dist_list:
            emb_path = make_embedding_filename(
                output_dir=embeddings_dir,
                traj_csv_path=traj_csv_path,
                seed=random_state,
                n_neighbors=nn,
                min_dist=md,
                metric=metric,
                n_samples=n_samples,
            )

            if os.path.exists(emb_path):
                print(f"[CACHE] Loading embedding from {emb_path}")
                emb = np.load(emb_path)
            else:
                print(f"[UMAP] n_neighbors={nn}, min_dist={md} (seed={random_state})")
                reducer = umap.UMAP(
                    n_neighbors=nn,
                    min_dist=md,
                    metric=metric,
                    n_components=2,
                    random_state=random_state,
                )
                emb = reducer.fit_transform(X)
                np.save(emb_path, emb)
                print(f"[SAVE] Stored embedding to {emb_path}")

            metrics = compute_quality_metrics(X, emb, n_neighbors=quality_k)

            embeddings[(nn, md)] = emb
            row = {
                "n_neighbors": nn,
                "min_dist": md,
                "metric": metric,
                "seed": random_state,
                "trustworthiness": metrics["trustworthiness"],
                "knn_overlap": metrics["knn_overlap"],
                "embedding_path": emb_path,
            }
            results.append(row)

    results_df = pd.DataFrame(results)
    return results_df, embeddings


In [ ]:
# Define the hyperparameter grid
n_neighbors_list = [5, 15, 50]
min_dist_list    = [0.0, 0.1, 0.5]

metric    = "euclidean"
quality_k = 15

results_df, embeddings = run_umap_grid(
    X,
    n_neighbors_list=n_neighbors_list,
    min_dist_list=min_dist_list,
    metric=metric,
    random_state=SEED,
    quality_k=quality_k,
    traj_csv_path=traj_csv_path,
    embeddings_dir=embeddings_output_dir,
)

# Sort by trustworthiness (descending) for quick inspection
results_df_sorted = results_df.sort_values("trustworthiness", ascending=False)
results_df_sorted


In [ ]:
def plot_embedding_grid(
    df_sub,
    embeddings,
    results_df,
    n_neighbors_list,
    min_dist_list,
    figsize_scale=4,
    plot_path=None,
):
    """
    Create a grid of scatterplots:
    rows   = different n_neighbors
    columns= different min_dist
    """
    if "line" not in df_sub.columns:
        raise SystemExit("Column 'line' not found in dataframe; needed for coloring.")

    cat = pd.Categorical(df_sub["line"])
    codes = cat.codes  # 0..K-1

    n_rows = len(n_neighbors_list)
    n_cols = len(min_dist_list)

    fig, axes = plt.subplots(
        n_rows,
        n_cols,
        figsize=(figsize_scale * n_cols, figsize_scale * n_rows),
        squeeze=False,
    )

    for i, nn in enumerate(n_neighbors_list):
        for j, md in enumerate(min_dist_list):
            ax = axes[i, j]
            key = (nn, md)
            if key not in embeddings:
                ax.axis("off")
                continue

            emb = embeddings[key]
            sc = ax.scatter(
                emb[:, 0],
                emb[:, 1],
                c=codes,
                s=3,
                alpha=0.7,
                cmap="tab20",
            )
            ax.set_xticks([])
            ax.set_yticks([])

            match = results_df[
                (results_df["n_neighbors"] == nn)
                & (results_df["min_dist"] == md)
            ]
            if not match.empty:
                row = match.iloc[0]
                title = (
                    f"nn={nn}, md={md}\n"
                    f"trust={row['trustworthiness']:.3f}, "
                    f"kNNov={row['knn_overlap']:.3f}"
                )
            else:
                title = f"nn={nn}, md={md}"
            ax.set_title(title, fontsize=9)

    fig.tight_layout()

    if plot_path is not None:
        fig.savefig(plot_path, dpi=300)

    return fig


base_traj = os.path.splitext(os.path.basename(traj_csv_path))[0]
grid_plot_path = os.path.join(
    embeddings_output_dir,
    f"{base_traj}_umap_grid_seed{SEED}.png",
)

fig = plot_embedding_grid(
    df_sub=df_sub,
    embeddings=embeddings,
    results_df=results_df,
    n_neighbors_list=n_neighbors_list,
    min_dist_list=min_dist_list,
    figsize_scale=4,
    plot_path=grid_plot_path,
)

print("Saved grid plot to:", grid_plot_path)


In [ ]:
# Pick the best config by trustworthiness
best = results_df.sort_values("trustworthiness", ascending=False).iloc[0]
best_key = (int(best["n_neighbors"]), float(best["min_dist"]))
best_emb = embeddings[best_key]

print("Best config:", best_key)
print("trustworthiness =", best["trustworthiness"], "kNN overlap =", best["knn_overlap"])

# Attach best embedding as x,y to df_sub and save
df_sub_best = df_sub.copy()
df_sub_best["x"] = best_emb[:, 0]
df_sub_best["y"] = best_emb[:, 1]

base_traj = os.path.splitext(os.path.basename(traj_csv_path))[0]
best_subset_path = os.path.join(
    embeddings_output_dir,
    f"{base_traj}_best_umap_subset_seed{SEED}.csv",
)

df_sub_best.to_csv(best_subset_path, index=False)
print("Saved best subset with embedding to:", best_subset_path)
